In [1]:
import torch
import pandas as pd

In [2]:
data = pd.read_csv('/content/fmnist_small.csv')


In [34]:
data.head()
data.dropna(inplace=True)

In [35]:
from torch.utils.data import Dataset


In [36]:
class CustomDataset(Dataset):
  def __init__(self,features,labels):
    self.features = torch.tensor(features,dtype=torch.float32)
    self.labels = torch.tensor(labels,dtype=torch.long)

  def __len__(self):
    return len(self.features)

  def __getitem__(self,idx):
    return self.features[idx],self.labels[idx]



In [37]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(data.drop('label',axis=1),data['label'],test_size=0.2,random_state=42)
X_train/=255.0
X_test/=255.0


In [38]:
train_dataset = CustomDataset(X_train.values,y_train.values)
test_dataset = CustomDataset(X_test.values,y_test.values)


In [39]:
from torch.utils.data import DataLoader
train_loader = DataLoader(train_dataset,batch_size=32,shuffle=True)
test_loader = DataLoader(test_dataset,batch_size=32, shuffle=True)


In [40]:
from torch import nn

In [41]:
class NeuralNetwork(nn.Module):
  def __init__(self,num_features):
    super().__init__()
    self.model = nn.Sequential(
        nn.Linear(num_features,128),
        nn.ReLU(),
        nn.Linear(128,64),
        nn.ReLU(),
        nn.Linear(64,10)
    )

  def forward(self,x):
    return self.model(x)



In [42]:
epochs = 100
learning_rate = 0.1
model = NeuralNetwork(X_train.shape[1])
criterion = nn.CrossEntropyLoss()


In [43]:
import torch.optim as optim

optimizer = optim.SGD(model.parameters(),lr=learning_rate)


In [45]:
for epoch in range(epochs):
    total_loss = 0
    for batch_features, batch_labels in train_loader:
        outputs = model(batch_features)
        loss = criterion(outputs, batch_labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch}, Average Loss: {avg_loss:.4f}")




Epoch 0, Average Loss: 0.0449
Epoch 1, Average Loss: 0.0299
Epoch 2, Average Loss: 0.0299
Epoch 3, Average Loss: 0.0245
Epoch 4, Average Loss: 0.0247
Epoch 5, Average Loss: 0.0315
Epoch 6, Average Loss: 0.0214
Epoch 7, Average Loss: 0.0264
Epoch 8, Average Loss: 0.1103
Epoch 9, Average Loss: 0.0615
Epoch 10, Average Loss: 0.0552
Epoch 11, Average Loss: 0.0233
Epoch 12, Average Loss: 0.0186
Epoch 13, Average Loss: 0.1782
Epoch 14, Average Loss: 0.0467
Epoch 15, Average Loss: 0.0344
Epoch 16, Average Loss: 0.0209
Epoch 17, Average Loss: 0.0174
Epoch 18, Average Loss: 0.0191
Epoch 19, Average Loss: 0.0204
Epoch 20, Average Loss: 0.2235
Epoch 21, Average Loss: 0.0173
Epoch 22, Average Loss: 0.0173
Epoch 23, Average Loss: 0.0154
Epoch 24, Average Loss: 0.5047
Epoch 25, Average Loss: 0.2453
Epoch 26, Average Loss: 0.0590
Epoch 27, Average Loss: 0.0452
Epoch 28, Average Loss: 0.0408
Epoch 29, Average Loss: 0.0208
Epoch 30, Average Loss: 0.1107
Epoch 31, Average Loss: 0.4098
Epoch 32, Average 

In [46]:
model.eval()

NeuralNetwork(
  (model): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=10, bias=True)
  )
)

In [53]:
total = 0
correct = 0
with torch.no_grad():
    for batch_features, batch_labels in test_loader:
        outputs = model(batch_features)
        _, predicted = torch.max(outputs, 1)
        total += batch_labels.shape[0]
        correct += (predicted == batch_labels).sum().item()

accuracy = correct / total
print(f"Accuracy:{accuracy*100:.2f}%")

Accuracy:82.27%
